# Google colab version

In [ ]:
# Clone the repository
# !git clone https://github.com/jonasricker/aeroblade.git

# Navigate into the cloned directory
%cd aeroblade

# Install a specific Python version using a more stable method for Colab
# Note: This will restart the kernel.
!pip install -q virtualenv
!virtualenv -p python3.10 aeroblade_env
!source aeroblade_env/bin/activate

c:\PersonalStuff\Monash\Sem3\FIT5230MaliciousAI\Assignments\cloned_repos\aeroblade


c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# Install the core and pip-managed packages.
# We are now pinning a compatible version of peft.
!pip install -v --no-cache-dir --no-input \
    --extra-index-url https://download.pytorch.org/whl/cu128 \
    torch==2.8.0+cu128 torchvision==0.23.0+cu128 \
    peft==0.7.0 \
    scipy numpy \
    scikit-learn scikit-image \
    seaborn tqdm \
    networkx pyarrow \
    diffusers==0.25.1 huggingface-hub==0.25.2 \
    requests PyYAML filelock \
    pillow opencv-python \
    jupyter-client ipykernel ipython jupyter-core \
    clip-interrogator==0.6.0 joblib==1.5.2 lpips==0.1.4 pyiqa==0.1.14.1
    
# Install the project in editable mode
!pip install -e .

# Locally run version. You may need to run this as administrator.

In [1]:
# # Clone the repository
# !git clone https://github.com/jonasricker/aeroblade.git

# # Navigate into the cloned directory
%cd aeroblade

c:\PersonalStuff\Monash\Sem3\FIT5230MaliciousAI\Assignments\cloned_repos\aeroblade


In [7]:
import sys
import subprocess
import shutil

# Find the full path to the conda executable
conda_path = shutil.which("conda")
if conda_path is None:
    print("Error: conda executable not found. Please ensure conda is installed and in your system PATH.")
    # You might want to exit the script here.
else:
    print(f"Found conda at: {conda_path}")

    # List of conda-managed packages
    conda_packages = [
        'python=3.10',
        'numpy=1.26.3',
        'pandas=2.1.4',
        'scipy=1.12.0',
        'matplotlib=3.8.2',
        'scikit-learn=1.4.0',
        'scikit-image=0.22.0',
        'seaborn=0.13.2',
        'tqdm=4.66.1',
        'networkx=3.2.1',
        'pyarrow=15.0.0',
        'pip'
    ]

    # Install conda packages
    print("Installing conda packages...")
    try:
        subprocess.run([conda_path, 'install', '-y', '-c', 'conda-forge'] + conda_packages, check=True)
        print("Successfully installed conda packages.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to install conda packages: {e}")

Found conda at: C:\Users\adria\miniconda3\condabin\conda.BAT
Installing conda packages...


In [8]:
import sys
import subprocess

# List of pip-managed packages
pip_packages = [
    '--extra-index-url', 'https://download.pytorch.org/whl/cu128',
    'torch==2.8.0+cu128',
    'torchvision==0.23.0+cu128',
    'diffusers==0.25.1',
    'huggingface-hub==0.25.2',
    'transformers==4.37.2',
    'requests==2.31.0',
    'PyYAML==6.0.1',
    'filelock==3.13.1',
    'pillow==10.2.0',
    'opencv-python==4.9.0.80',
    'jupyter-client==8.6.0',
    'ipykernel==6.28.0',
    'ipython==8.20.0',
    'jupyter-core==5.7.1',
    'clip-interrogator==0.6.0',
    'joblib==1.5.2',
    'lpips==0.1.4',
    'pyiqa==0.1.14.1'
]

# Install pip packages
print("Installing pip packages...")
try:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', '--no-input'] + pip_packages, check=True)
    print("Successfully installed pip packages.")
except subprocess.CalledProcessError as e:
    print(f"Failed to install pip packages: {e}")

# Replicates !pip install -e . Might need administrator access.
print("Installing in editable mode...")
try:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], check=True)
    print("Successfully installed in editable mode.")
except subprocess.CalledProcessError as e:
    print(f"Failed to install in editable mode: {e}")

Installing pip packages...
Installing in editable mode...


# Run the scripts. 
This is the same whether on colab or locally run.
Input your adversarial images as png images into example_images, or specify the folder directory manually.

In [ ]:
### SET IMAGE DIRECTORY HERE
IMG_DIR = "real_img/real_1024"

In [ ]:
# Preprocess images with vaeround - increases robustness to generated images
import os
from PIL import Image

VAEROUND_DIR = f"{IMG_DIR}/vaeround"

def list_images(input_dir):
    exts = ('.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff')
    for root, _, files in os.walk(input_dir):
        for f in files:
            if f.lower().endswith(exts):
                yield os.path.join(root, f)

def save_img(img: Image.Image, src_path, out_root, transform_name, input_root):
    rel = os.path.relpath(src_path, start=input_root)   # key fix
    dest_dir = os.path.join(out_root, transform_name, os.path.dirname(rel))
    os.makedirs(dest_dir, exist_ok=True)
    dest_path = os.path.join(dest_dir, os.path.basename(rel))
    img.save(dest_path, quality=95)
    return dest_path


def vae_roundtrip_pil(img: Image.Image, vae=None, device="cpu"):
    """
    Convert PIL image -> latent via SD VAE encoder -> decode back.
    vae should be an AutoencoderKL or compatible model from diffusers.
    Works at 1024 resolution expected; will resize image to 1024x1024.
    """
    if vae is None:
        raise RuntimeError("VAE model not provided")
    import torch
    from torchvision import transforms
    img = img.convert("RGB")
    img_resized = img.resize((1024, 1024), resample=Image.BICUBIC)
    to_tensor = transforms.ToTensor()
    x = to_tensor(img_resized).unsqueeze(0).to(device) * 2.0 - 1.0  # [-1,1]
    with torch.no_grad():
        latent = vae.encode(x).latent_dist.sample() * vae.scaling_factor
        decoded = vae.decode(latent / vae.scaling_factor).sample
    decoded = (decoded / 2 + 0.5).clamp(0, 1)
    decoded = (decoded[0].cpu().permute(1, 2, 0).numpy() * 255).astype('uint8')
    return Image.fromarray(decoded)

inp = IMG_DIR
out = IMG_DIR

vae = None
try:
    from diffusers import AutoencoderKL
    # choose a VAE checkpoint compatible with SD. Adjust repo/model as needed.
    vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to("cuda")
    vae.eval()
    print("Loaded VAE.")
except Exception as e:
    print("Failed to load VAE (skipping). Exception:", e)
    vae = None

for path in list_images(inp):
    try:
        img = Image.open(path).convert('RGB')
    except Exception as e:
        print("skip", path, e)
        continue
    
    # VAE roundtrip    
    try:
        vr = vae_roundtrip_pil(img, vae=vae, device="cuda")
        save_img(vr, path, out, "vaeround", inp)
    except Exception as e:
        print("vae roundtrip failed for", path, e)

print("Done. Preprocessed folders in:", out)

In [ ]:
# Image directory specified above
# !python scripts/run_aeroblade.py --files-or-dirs path/to/img.png path/to/dir

%run scripts/run_aeroblade.py --files-or-dirs {VAEROUND_DIR} #real_img/real_1024_processed/vaeround

# # Runs on the images in example_images
# %run scripts/run_aeroblade.py 

c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\diffusers\utils\outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\diffusers\utils\outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
PROGRESS (compute_distances):   0%|          | 0/3 [00:00<?, ?it/s]c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_down

Images saved to aeroblade_output\reconstructions\643235ac5ed4bcd82abc3917c198526e\1.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\lpips\weights\v0.1\vgg.pth


Computing LPIPS: 100%|██████████| 500/500 [02:00<00:00,  4.16it/s]
c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\joblib\_store_backends.py:226: CacheWarning: Unable to cache to disk. Possibly a race condition in the creation of the directory. Exception: LZ4 is not installed. Install it with pip: https://python-lz4.readthedocs.io/.
  warnings.warn(
PROGRESS (compute_distances):  33%|███▎      | 1/3 [14:04<28:09, 844.71s/it]c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Reconstructing with stabilityai/stable-diffusion-2-base.: 100%|██████████| 500/500 [11:40<00:00,  1.40s/it]
C:\PersonalStuff\Monash\Sem3\FIT5230MaliciousAI\Assignments\cloned_repos\aeroblade\src\aeroblade\distances.py:40: UserWarning: ds_a and ds_b contain f

Images saved to aeroblade_output\reconstructions\5a19b4385f1c55a314753b37f685d7bc\1.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\lpips\weights\v0.1\vgg.pth


Computing LPIPS: 100%|██████████| 500/500 [01:57<00:00,  4.25it/s]
c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\joblib\_store_backends.py:226: CacheWarning: Unable to cache to disk. Possibly a race condition in the creation of the directory. Exception: LZ4 is not installed. Install it with pip: https://python-lz4.readthedocs.io/.
  warnings.warn(
PROGRESS (compute_distances):  67%|██████▋   | 2/3 [27:45<13:50, 830.91s/it]c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type M-CLIP. This is not supported for all configurations of models and can yield errors.
Reconstructing with kandinsky-community/kandinsky-2-1.: 100%|██████████| 500/500 [15:42<00:00,  1.89

Images saved to aeroblade_output\reconstructions\67252c6648f501f03984100cb7f348b6\1.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\lpips\weights\v0.1\vgg.pth


Computing LPIPS: 100%|██████████| 500/500 [01:56<00:00,  4.29it/s]
c:\Users\adria\miniconda3\envs\aeroblade\lib\site-packages\joblib\_store_backends.py:226: CacheWarning: Unable to cache to disk. Possibly a race condition in the creation of the directory. Exception: LZ4 is not installed. Install it with pip: https://python-lz4.readthedocs.io/.
  warnings.warn(
PROGRESS (compute_distances): 100%|██████████| 3/3 [45:40<00:00, 913.44s/it]

                                        dir  image_size  \
0     real_img\real_1024_processed\vaeround        1024   
1     real_img\real_1024_processed\vaeround        1024   
2     real_img\real_1024_processed\vaeround        1024   
3     real_img\real_1024_processed\vaeround        1024   
4     real_img\real_1024_processed\vaeround        1024   
...                                     ...         ...   
1995  real_img\real_1024_processed\vaeround        1024   
1996  real_img\real_1024_processed\vaeround        1024   
1997  real_img\real_1024_processed\vaeround        1024   
1998  real_img\real_1024_processed\vaeround        1024   
1999  real_img\real_1024_processed\vaeround        1024   

                            repo_id transform distance_metric      file  \
0     CompVis/stable-diffusion-v1-1     clean     lpips_vgg_2  0000.jpg   
1     CompVis/stable-diffusion-v1-1     clean     lpips_vgg_2  0001.jpg   
2     CompVis/stable-diffusion-v1-1     clean     lpips_vgg_2  000

In [ ]:
import pandas as pd
import os

# Input / output paths
input_csv = "aeroblade_output/distances.csv"
output_csv = "aeroblade_output/predictions.csv"

# Load distances
df = pd.read_csv(input_csv)

# Keep only 'max' rows (final aggregated distance per image)
df_max = df[df["repo_id"] == "max"].copy()

# Classification rule
def classify(distance):
    if 0 >= distance >= -0.001:
        return "fake"
    elif distance < -0.001:
        return "true"
    else:
        return "unknown"

df_max["prediction"] = df_max["distance"].apply(classify)

# Extract ground-truth labels from dir or file
def extract_label(path, filename):
    path_str = f"{path} {filename}".lower()
    if "real" in path_str:
        return "true"
    elif "fake" in path_str:
        return "fake"
    else:
        return "unknown"

df_max["ground_truth"] = df_max.apply(
    lambda row: extract_label(row["dir"], row["file"]), axis=1
)

# Select requested columns
df_pred = df_max[["dir", "image_size", "file", "prediction", "ground_truth"]]

# Save predictions
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
df_pred.to_csv(output_csv, index=False)
print(f"Saved predictions to {output_csv}")

# --- Summary stats ---
# Count predictions overall
counts = df_pred["prediction"].value_counts(dropna=False)
print("\nPrediction counts (all images):")
print(counts.to_string())

# Evaluate only where ground-truth is known
valid = df_pred[df_pred["ground_truth"] != "unknown"]
if not valid.empty:
    accuracy = (valid["prediction"] == valid["ground_truth"]).mean()
    print(f"\nAccuracy (on {len(valid)} images with ground truth): {accuracy:.4f}")

    # Confusion matrix
    confusion = pd.crosstab(valid["ground_truth"], valid["prediction"],
                            rownames=["Actual"], colnames=["Predicted"], dropna=False)
    print("\nConfusion Matrix (only known ground truths):")
    print(confusion)
else:
    print("\n⚠️ No ground-truth labels detected (check filenames or directories).")


Saved predictions to aeroblade_output/predictions.csv

Prediction counts (all images):
prediction
true    15

Accuracy (on 15 images with ground truth): 0.0000

Confusion Matrix (only known ground truths):
Predicted  true
Actual         
fake         15
